In [1]:
import numpy as np
import pandas as pd
import tensorflow.compat.v1 as tf
from koopmanlib.dictionary import PsiNN
from koopmanlib.solver import KoopmanDLSolver

2025-06-09 03:37:18.370570: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-09 03:37:18.385789: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-06-09 03:37:18.403348: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-06-09 03:37:18.408450: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-09 03:37:18.421401: I tensorflow/core/platform/cpu_feature_guar

In [2]:
dim = 3 
hidden_dim = 40
input_dim = 0
n_feature = 2
num_epochs = 20
batch_size = 512
n_train = 10000
n_valid = 1000
n_test = 1000
learning_rate = 1e-3

In [3]:
X_train = pd.read_csv('KO_sample_train.csv', header=None).values
X_valid = pd.read_csv('KO_sample_valid.csv', header=None).values
X_test = pd.read_csv('KO_sample_test.csv', header=None).values

length = X_train.shape[1] // n_train
H_train = []
for i in range(n_train):
    H_train.append(X_train[:, i*length:(i+1)*length])
H_train = np.stack([H_train[idx].T for idx in range(n_train)], axis=0)
H_valid = []
for i in range(n_valid):
    H_valid.append(X_valid[:, i*length:(i+1)*length])
H_valid = np.stack([H_valid[idx].T for idx in range(n_valid)], axis=0)
H_test = []
for i in range(n_test):
    H_test.append(X_test[:, i*length:(i+1)*length])
H_test = np.stack([H_test[idx].T for idx in range(n_test)], axis=0)

In [ ]:
sample_list = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
error_list = []

for i in sample_list:
    error_temp = []
    data_train = [H_train[:, 0:10*i:i,:].reshape(-1, dim+input_dim), H_train[:, i:11*i:i,:].reshape(-1, dim+input_dim)]
    data_valid = [H_valid[:, 0:10*i:i,:].reshape(-1, dim+input_dim), H_valid[:, i:11*i:i,:].reshape(-1, dim+input_dim)]
    basis_function = PsiNN(layer_sizes=[hidden_dim, hidden_dim, hidden_dim], n_psi_train=dim+n_feature)
    solver = KoopmanDLSolver(dic=basis_function,
                             target_dim=dim+input_dim,
                             reg=0.0)
    solver.build(data_train=data_train,
                 data_valid=data_valid,
                 epochs=num_epochs,
                 batch_size=batch_size,
                 lr=learning_rate,
                 log_interval=1,
                 lr_decay_factor=0.92)
    for j in range(n_test):
        x_traj = H_test[j, 0:11*i:i,:]
        x0_test = x_traj[0]
        x0_test = x0_test.reshape(-1, x0_test.shape[-1])
        x_est_traj_DL = solver.predict(x0_test, 10)
        error_temp.append((x_est_traj_DL[:, :dim]-x_traj[1:, :dim]) ** 2)
    error_list.append(np.mean(np.array(error_temp)))
    print(f"{i} done!")
    
df = pd.DataFrame(error_list)
df.to_csv("KO_sample.csv", index=False)

In [14]:
len(error_temp)

1000

In [15]:
error_list

[5.962117771016133e-06]